In [1]:
from pathlib import Path
queries=['O93430','P23415','O75311'] #UniPortKB ID
mer=5 #Total number of chains
cwd=Path("glyr_test") #This directory must exist
cwd.mkdir(exist_ok=True)
exclude=['2M6I'] #[List of PDB IDs] 
#If you already know some structures you want to exclude provide them as a list. 

In [2]:
from enspdb.prep_ensemble import PDBInfo
info=PDBInfo(queries,mer,exclude=exclude) #Add the exclude argument to the end.

CRITICAL:root:Cannot process PDB id 1MOT. It does not contain complete set
CRITICAL:root:Cannot process PDB id 1VRY. It does not contain complete set
CRITICAL:root:Cannot process PDB id 2M6B. It does not contain complete set


In [3]:
info.query #UniPort ID used to initialize this class
#and it is also used for the output names.
print(info.mer) # Total number of chains
print(info.result) #Contains the information from the 3D structures table.
print(info.refseqs) #Contains the reference sequence. 

5
{'3JAD': [1, [['A', 'B', 'C', 'D', 'E']]], '3JAE': [1, [['A', 'B', 'C', 'D', 'E']]], '3JAF': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLO': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLP': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLQ': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLR': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLS': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLT': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLU': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLV': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLW': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLX': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLY': [1, [['A', 'B', 'C', 'D', 'E']]], '6PLZ': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM0': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM1': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM2': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM3': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM4': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM5': [1, [['A', 'B', 'C', 'D', 'E']]], '6PM6': [1, [['A', 'B', 'C', 'D', 'E']]], '6PXD': [1, [['A', 'B', 'C', 'D', 'E']]], '6UBS': [1, [['A', 'B', 'C', 'D

In [4]:
info.downloadPDB(cwd)
info.process_pdbs(cwd)

CRITICAL:root:PDB ID 4X5T cannot be processed, possibly a chimera, skipping this file


In [5]:
clustalopath="/home/ozyo/.local/bin/clustalo-1.2.4-Ubuntu-x86_64"
r=info.msa_clustal(cwd,clustalopath)

In [6]:
info.get_core(cwd)

In [7]:
from MDAnalysis import Universe
from MDAnalysis.analysis import align
import pandas as pd

def align_get_rmsd(ref_id:str,cwd:Path):
    ref = Universe(f"{cwd}/clean_pdbs/correct_{ref_id}.pdb")
    data = {"pdb_name":[],"rmsd":[],"universes":[]}
    for pdb in info.coremer:
        if pdb in info.broken:
            continue
        u=Universe(f"{cwd}/clean_pdbs/correct_{pdb}")
        a = align.AlignTraj(u,ref,in_memory=True)
        a.run()
        data["pdb_name"].append(pdb)
        data["universes"].append(u)
        data["rmsd"].append(a.results.rmsd[0])
    data=pd.DataFrame(data)
    print(data[data.rmsd > 5])
    return data

rmsd=align_get_rmsd('3JAD_1',cwd)

/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


      pdb_name       rmsd                   universes
1   6PLR_1.pdb  33.607393  <Universe with 1645 atoms>
2   6PM0_1.pdb  33.807739  <Universe with 1645 atoms>
3   6PM5_1.pdb  33.577508  <Universe with 1645 atoms>
4   6UD3_1.pdb  25.217131  <Universe with 1645 atoms>
7   6PLX_1.pdb  33.569200  <Universe with 1645 atoms>
8   6PLQ_1.pdb  33.654846  <Universe with 1645 atoms>
9   6VM3_1.pdb  25.227764  <Universe with 1645 atoms>
10  5TIO_1.pdb  24.998971  <Universe with 1645 atoms>
12  6PLS_1.pdb  33.622289  <Universe with 1645 atoms>
13  6VM0_1.pdb  25.206892  <Universe with 1645 atoms>
14  6PM1_1.pdb  33.539067  <Universe with 1645 atoms>
15  6PLW_1.pdb  33.818750  <Universe with 1645 atoms>
16  6PM3_1.pdb  33.242504  <Universe with 1645 atoms>
18  6PLP_1.pdb  33.654846  <Universe with 1645 atoms>
20  6PM4_1.pdb  33.584811  <Universe with 1645 atoms>
22  6PLY_1.pdb  33.620690  <Universe with 1645 atoms>
23  6PLO_1.pdb  33.549044  <Universe with 1645 atoms>
25  6VM2_1.pdb  25.206892  <

As you can see in the above analysis we have quite a few PDB files that show a high RMSD difference to the reference. They are outliers and most of the time you have to examine what's wrong manually in each file and either discard or repair them. GLIC has 5 subunits arranged in counter-clockwise order, usually, however, the PDB files above do not contain chains in the counter-clockwise order and their order needs to be changed.

In [9]:
# run it twice to fix list a
from enspdb import chorder
from importlib import reload
reload(chorder)
b=rmsd[rmsd.rmsd>5].pdb_name.to_list()
a= ['6VM2_1.pdb','6VM3_1.pdb','6VM0_1.pdb','6UD3_1.pdb','5TIO_1.pdb']
forced ={k:['A','E','D','C','B'] for k in b if k not in a}
for k in a:
    forced[k]=['A','D','C','B','E']
chorder.forceorder({k:v for k,v in info.coremer.items() if k not in info.broken},cwd/"clean_pdbs",tag="correct_", alph=False,forordict=forced)
rmsd = align_get_rmsd('3JAD_1',cwd)

/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


Empty DataFrame
Columns: [pdb_name, rmsd, universes]
Index: []


In [10]:
for _,row in rmsd.iterrows():
    row['universes'].select_atoms("all").write(f"{cwd}/clean_pdbs/correct_{row['pdb_name']}")

/home/ozyo/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


In [11]:
%%bash
touch glyr_test/GLYR_ensemble.pdb
for pdb in `ls -1 glyr_test/clean_pdbs/correct*pdb`
do
echo "MODEL" >> glyr_test/GLYR_ensemble.pdb
cat $pdb >> glyr_test/GLYR_ensemble.pdb
echo "END" >> glyr_test/GLYR_ensemble.pdb
done